# GxE problem

In [ ]:
library(brms)
library(data.table)
library(texreg)

In [2]:
# Domingue's simulation function
sim.data.dom = function(E, b0 = 0.1, b1=0.1, b2=0.2, b3=.2, h=0.5, a = 0.2, sigma=1, scaling = TRUE) {
    N <- length(E)
    G <- rnorm(N,0,1)
    eps <- rnorm(N,0,sigma)
    if (scaling){
        e <- sqrt(1-h^2)
        ystar <- h*G + e*eps
        y <- a*E+(b0 + b1*E)*ystar
    } else {
        y <-  b1*G+b2*E+b3*G*E+eps*E+eps
    }
    if (scaling) {
        df <- data.frame(E = E, y = y, g = G, ystar = ystar, h = h, eps = eps,  e = e)
    }
    else {
        df <- data.frame(E  = E, y = y, g = G, eps = eps)
    }
    return(df)
}

## Scaling model

In [3]:
E = rnorm(5000, 0, 1)
dts = data.table(sim.data.dom(E, scaling = TRUE))
dts[, sd := e * eps]
summary(dts)

       E                  y                   g                ystar          
 Min.   :-3.51315   Min.   :-0.934071   Min.   :-3.52681   Min.   :-3.419104  
 1st Qu.:-0.66282   1st Qu.:-0.161978   1st Qu.:-0.65700   1st Qu.:-0.682363  
 Median : 0.01408   Median :-0.039251   Median :-0.01018   Median :-0.021061  
 Mean   : 0.01492   Mean   : 0.001047   Mean   : 0.00145   Mean   :-0.006696  
 3rd Qu.: 0.68616   3rd Qu.: 0.129512   3rd Qu.: 0.64311   3rd Qu.: 0.669643  
 Max.   : 3.66642   Max.   : 1.341220   Max.   : 3.96468   Max.   : 3.785653  
       h            eps                  e               sd           
 Min.   :0.5   Min.   :-3.718512   Min.   :0.866   Min.   :-3.220326  
 1st Qu.:0.5   1st Qu.:-0.694542   1st Qu.:0.866   1st Qu.:-0.601491  
 Median :0.5   Median :-0.014119   Median :0.866   Median :-0.012228  
 Mean   :0.5   Mean   :-0.008569   Mean   :0.866   Mean   :-0.007421  
 3rd Qu.:0.5   3rd Qu.: 0.676163   3rd Qu.:0.866   3rd Qu.: 0.585574  
 Max.   :0.5   Max.  

In [4]:
cnames = c("ystar", "y", "y-wls")
m1 = lm(ystar ~ g + E + g * E, data = dts)
m2 = lm(y ~ g + E + g * E, data = dts)
wts = 1 / fitted(lm(abs(residuals(m2)) ~ fitted(m2)))^2
m3  = lm(y ~ g + E + g * E, data = dts, weights=wts)
cat(screenreg(list(m1, m2, m3)))


             Model 1      Model 2      Model 3    
--------------------------------------------------
(Intercept)    -0.01        -0.00         0.03 *  
               (0.01)       (0.00)       (0.02)   
g               0.51 ***     0.05 ***     0.36 ***
               (0.01)       (0.00)       (0.01)   
E              -0.01         0.20 ***     0.22 ***
               (0.01)       (0.00)       (0.01)   
g:E            -0.01         0.05 ***     0.20 ***
               (0.01)       (0.00)       (0.00)   
--------------------------------------------------
R^2             0.25         0.74         0.44    
Adj. R^2        0.25         0.74         0.43    
Num. obs.    5000         5000         5000       
*** p < 0.001; ** p < 0.01; * p < 0.05



# Bayesian distributional model

In [ ]:
# distributional model using bayesian stats

f = bf(y ~ g + E + g * E, sigma ~ 1 + E)
m4 = brm(f, data = dts)

In [6]:
summary(m4)

 Family: gaussian 
  Links: mu = identity; sigma = log 
Formula: y ~ g + E + g * E 
         sigma ~ 1 + E
   Data: dts (Number of observations: 5000) 
Samples: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup samples = 4000

Population-Level Effects: 
                Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept          -0.00      0.00    -0.01     0.00 1.00     4726     3044
sigma_Intercept    -2.27      0.01    -2.29    -2.25 1.00     1922     2276
g                   0.05      0.00     0.05     0.05 1.00     4803     2981
E                   0.20      0.00     0.19     0.20 1.00     4784     2996
g:E                 0.05      0.00     0.05     0.05 1.00     4975     2723
sigma_E             0.32      0.01     0.31     0.33 1.00     1907     1657

Samples were drawn using sampling(NUTS). For each parameter, Bulk_ESS
and Tail_ESS are effective sample size measures, and Rhat is the potential
scale reduction factor on split c

In [19]:
hyp <- "g * exp(sigma_E) = g:E * exp(sigma_Intercept)"
(hyp <- hypothesis(m4, hyp, alpha = 0.05))


Hypothesis Tests for class b:
                Hypothesis Estimate Est.Error CI.Lower CI.Upper Evid.Ratio
1 (g*exp(sigma_E))-... = 0     0.06         0     0.06     0.07         NA
  Post.Prob Star
1        NA    *
---
'CI': 90%-CI for one-sided and 95%-CI for two-sided hypotheses.
'*': For one-sided hypotheses, the posterior probability exceeds 95%;
for two-sided hypotheses, the value tested against lies outside the 95%-CI.
Posterior probabilities of point hypotheses assume equal prior probabilities.